In [3]:
import pandas as pd

# Path to the Excel files
file_path = r'C:\Users\Svitlana\OneDrive\RoFacts\mvp\Raw Data\\'

# List of file names
file_names = ["inflation.xlsx", "average_wage.xlsx"]

# Dictionary to hold DataFrames
dfs = {}
# Read each Excel file
for file_name in file_names:
    try:
        df = pd.read_excel(file_path + file_name)
        dfs[file_name.split('.')[0]] = df  # Store DataFrame with file name as key
        print(f"Data from '{file_name}' read successfully.")
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")
    except Exception as e:
        print(f"An error occurred while reading '{file_name}':", e)

# Example of accessing DataFrames
if 'inflation' in dfs:
    print("\nData from 'inflation' DataFrame:")
    print(dfs['inflation'].head())

if 'average_wage' in dfs:
    print("\nData from 'average_wage' DataFrame:")
    print(dfs['average_wage'].head())

Data from 'inflation.xlsx' read successfully.
Data from 'average_wage.xlsx' read successfully.

Data from 'inflation' DataFrame:
   Year  Inflation
0  1971      0.006
1  1972      0.000
2  1973      0.007
3  1974      0.011
4  1975      0.002

Data from 'average_wage' DataFrame:
   Year     Jan     Feb     Mar     Apr     Maz     Jun     Jul     Aug  \
0  1991    4038    3978    4110    6213    7550    7787    8263    8239   
1  1992   13005   12717   15287   15677   17709   19426   19989   19804   
2  1993   27763   29162   37146   38319   51034   58917   67047   75032   
3  1994  101795  106378  112603  126194  126464  131134  142657  153883   
4  1995  170885  173758  182803  199030  199702  205080  218535  230338   

      Sep     Oct     Nov     Dec     Yearly Avg  Average wage (RON)  
0    9174    9824   10773   11824    7647.750000            0.764775  
1   23306   24080   28456   32612   20172.333333            2.017233  
2   74723   79732   93020  101331   61102.166667        

In [4]:
print(dfs['average_wage'].head())

   Year     Jan     Feb     Mar     Apr     Maz     Jun     Jul     Aug  \
0  1991    4038    3978    4110    6213    7550    7787    8263    8239   
1  1992   13005   12717   15287   15677   17709   19426   19989   19804   
2  1993   27763   29162   37146   38319   51034   58917   67047   75032   
3  1994  101795  106378  112603  126194  126464  131134  142657  153883   
4  1995  170885  173758  182803  199030  199702  205080  218535  230338   

      Sep     Oct     Nov     Dec     Yearly Avg  Average wage (RON)  
0    9174    9824   10773   11824    7647.750000            0.764775  
1   23306   24080   28456   32612   20172.333333            2.017233  
2   74723   79732   93020  101331   61102.166667            6.110217  
3  153486  160483  167745  198530  140112.666667           14.011267  
4  229543  242612  252217  282995  215624.833333           21.562483  


In [5]:
inflation_df = dfs['inflation']
wages_df = dfs['average_wage']

print(inflation_df.head())
print(wages_df.head())

   Year  Inflation
0  1971      0.006
1  1972      0.000
2  1973      0.007
3  1974      0.011
4  1975      0.002
   Year     Jan     Feb     Mar     Apr     Maz     Jun     Jul     Aug  \
0  1991    4038    3978    4110    6213    7550    7787    8263    8239   
1  1992   13005   12717   15287   15677   17709   19426   19989   19804   
2  1993   27763   29162   37146   38319   51034   58917   67047   75032   
3  1994  101795  106378  112603  126194  126464  131134  142657  153883   
4  1995  170885  173758  182803  199030  199702  205080  218535  230338   

      Sep     Oct     Nov     Dec     Yearly Avg  Average wage (RON)  
0    9174    9824   10773   11824    7647.750000            0.764775  
1   23306   24080   28456   32612   20172.333333            2.017233  
2   74723   79732   93020  101331   61102.166667            6.110217  
3  153486  160483  167745  198530  140112.666667           14.011267  
4  229543  242612  252217  282995  215624.833333           21.562483  


In [6]:
columns_to_keep = ['Year', 'Average wage (RON)']

# Drop all the month columns except 'Year' and 'Average wage (RON)'
wages_df = wages_df[columns_to_keep]

# Display the updated DataFrame
print(wages_df.head())

print(inflation_df.head())

   Year  Average wage (RON)
0  1991            0.764775
1  1992            2.017233
2  1993            6.110217
3  1994           14.011267
4  1995           21.562483
   Year  Inflation
0  1971      0.006
1  1972      0.000
2  1973      0.007
3  1974      0.011
4  1975      0.002


In [7]:
# Merge the two data frame starting with year 1991

inflation_df = inflation_df[inflation_df['Year'] >= 1991]

merged_df = pd.merge(wages_df, inflation_df, on='Year', how='left')

print(merged_df.head())

   Year  Average wage (RON)  Inflation
0  1991            0.764775      1.702
1  1992            2.017233      2.104
2  1993            6.110217      2.561
3  1994           14.011267      1.367
4  1995           21.562483      0.323


In [8]:
# Drop the index column
merged_df = merged_df.reset_index(drop=True)

# Display the resulting DataFrame
print(merged_df.head())

   Year  Average wage (RON)  Inflation
0  1991            0.764775      1.702
1  1992            2.017233      2.104
2  1993            6.110217      2.561
3  1994           14.011267      1.367
4  1995           21.562483      0.323


In [9]:
# Transform Inflation to %

merged_df['Inflation'] = merged_df['Inflation'] * 100

In [10]:
# Set the inflation coefficient to 1 for 1991 and calcult CPI starting from that year
merged_df['CPI-1991'] = 0

merged_df.loc[0, 'CPI-1991'] = 1

# Calculate CPI for subsequent years
for i in range(1, len(merged_df)):
    inflation = merged_df.loc[i, 'Inflation']
    previous_cpi = merged_df.loc[i - 1, 'CPI-1991']
    merged_df.loc[i, 'CPI-1991'] = (1 + (inflation / 100)) * previous_cpi
    


print(merged_df)

    Year  Average wage (RON)  Inflation     CPI-1991
0   1991            0.764775      170.2     1.000000
1   1992            2.017233      210.4     3.104000
2   1993            6.110217      256.1    11.053344
3   1994           14.011267      136.7    26.163265
4   1995           21.562483       32.3    34.614000
5   1996           32.198683       38.8    48.044232
6   1997           63.682142      154.8   122.416703
7   1998          107.389808       59.1   194.764974
8   1999          155.473650       45.8   283.967332
9   2000          217.347808       45.7   413.740403
10  2001          305.359775       34.5   556.480843
11  2002          388.117833       22.5   681.689032
12  2003          486.810350       15.3   785.987454
13  2004          596.528517       11.9   879.519961
14  2005          737.833333        9.0   958.676758
15  2006          862.000000        6.6  1021.949424
16  2007         1042.916667        4.8  1071.002996
17  2008         1282.250000        7.9  1155.

C:\Users\Svitlana\AppData\Local\Temp\ipykernel_35948\1877031172.py:10: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.104' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [11]:
# Set the inflation coefficient to 1 for 1991 and calcult CPI starting from that year
merged_df['CPI-2023'] = 0

merged_df.loc[len(df) -1, 'CPI-2023'] = 1

# Calculate CPI for subsequent years
for i in range(len(merged_df) - 2, -1, -1):
    following_year_cpi = merged_df.loc[i + 1, 'CPI-2023']
    inflation_following_year = merged_df.loc[i + 1, 'Inflation']
    merged_df.loc[i, 'CPI-2023'] = following_year_cpi / ((1 + inflation_following_year / 100))
    


print(merged_df)

    Year  Average wage (RON)  Inflation     CPI-1991  CPI-2023
0   1991            0.764775      170.2     1.000000  0.000461
1   1992            2.017233      210.4     3.104000  0.001430
2   1993            6.110217      256.1    11.053344  0.005092
3   1994           14.011267      136.7    26.163265  0.012053
4   1995           21.562483       32.3    34.614000  0.015947
5   1996           32.198683       38.8    48.044232  0.022134
6   1997           63.682142      154.8   122.416703  0.056397
7   1998          107.389808       59.1   194.764974  0.089728
8   1999          155.473650       45.8   283.967332  0.130823
9   2000          217.347808       45.7   413.740403  0.190609
10  2001          305.359775       34.5   556.480843  0.256369
11  2002          388.117833       22.5   681.689032  0.314052
12  2003          486.810350       15.3   785.987454  0.362102
13  2004          596.528517       11.9   879.519961  0.405192
14  2005          737.833333        9.0   958.676758  0

C:\Users\Svitlana\AppData\Local\Temp\ipykernel_35948\1443100621.py:10: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9057971014492753' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [12]:
#rename merged_df to df

df = merged_df


# Calculate the percentage change in the 'Average Wage (RON)' column
df['Real Average Wage (RON) - 2023 prices'] = df['Average wage (RON)'] / df['CPI-2023']

# Calculate the percentage change in the 'Average Wage (RON)' column
df['Real Average Wage (RON) - 2023 prices'] = df['Real Average Wage (RON) - 2023 prices'].astype(int)


# Display the resulting DataFrame
print(df)

# Save the resulting DataFrame to a new csv file
output_file_path = r'C:\Users\Svitlana\OneDrive\RoFacts\mvp\Data\real_wage.csv'
df.to_csv(output_file_path, index=False)

    Year  Average wage (RON)  Inflation     CPI-1991  CPI-2023  \
0   1991            0.764775      170.2     1.000000  0.000461   
1   1992            2.017233      210.4     3.104000  0.001430   
2   1993            6.110217      256.1    11.053344  0.005092   
3   1994           14.011267      136.7    26.163265  0.012053   
4   1995           21.562483       32.3    34.614000  0.015947   
5   1996           32.198683       38.8    48.044232  0.022134   
6   1997           63.682142      154.8   122.416703  0.056397   
7   1998          107.389808       59.1   194.764974  0.089728   
8   1999          155.473650       45.8   283.967332  0.130823   
9   2000          217.347808       45.7   413.740403  0.190609   
10  2001          305.359775       34.5   556.480843  0.256369   
11  2002          388.117833       22.5   681.689032  0.314052   
12  2003          486.810350       15.3   785.987454  0.362102   
13  2004          596.528517       11.9   879.519961  0.405192   
14  2005  

In [ ]:
import plotly.graph_objects as go

# Define background color
color_background = '#F8EFDE'
color_wage = '#46C07a'
color_arrow = '#c46e6e'

# Assuming 'merged_df' is your DataFrame with the required columns

# Create a figure
fig = go.Figure()

# Add a line trace for Wage Growth
fig.add_trace(go.Scatter(x=merged_df['Year'], y=merged_df['Real Average Wage (RON) - 2023 prices'],
                         mode='lines+markers',
                         name='Wage Growth (%)',
                         line=dict(color=color_wage)))


# Add annotation for the important event
fig.add_annotation(x=2005, y=merged_df.loc[merged_df['Year'] == 2005, 'Real Average Wage (RON) - 2023 prices'].iloc[0],
                   text="The average real wage reached the 1991 level", showarrow=True,
                   font=dict(color= "black", size=12),
                   arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor=color_arrow,
                   ax=-200, ay=-200)

# Update layout
fig.update_layout(
    title='Real average wage (1991-2023)',
    xaxis_title='Year',
    yaxis_title='Amount (RON)',
    plot_bgcolor=color_background,
    paper_bgcolor=color_background,
    xaxis=dict(
        dtick=2
    ),  # Adjust the tick frequency here (e.g., 1 for every year)
    yaxis=dict(
        tickformat='.',  # Add thousands separator to y-axis
    )
)

# Add custom hover text for the point corresponding to 2005
hover_text = [f'{year}: {int(wage)}' + (' - The average real wage reached the 1991 level for the first time' if year == 2005 else '') for year, wage in zip(df['Year'], df['Real Average Wage (RON) - 2023 prices'])]

# Update hover text
fig.update_traces(hovertext=hover_text, hoverinfo='text')
# Show the plot
fig.show()


In [ ]:

# Calculate percentage change
df['Real Average Wage % change'] = df['Real Average Wage (RON) - 2023 prices'].pct_change() * 100

# Drop the NaN value in the first row (since there's no previous year to compare with)
#df.dropna(subset=['Average real  wage (% change)'], inplace=True)

df['Real Average Wage % change'] = df['Real Average Wage % change'].round(2)

print(df.head())



#Add a text box

import plotly.graph_objects as go

# Define background color
color_background = '#F8EFDE'
color_positive = '#46C07a'
color_negative = '#c46e6e'


# Create a figure
fig = go.Figure()

# Add a bar trace for Real Income
fig.add_trace(go.Bar(x=merged_df['Year'], y=df['Real Average Wage % change'],
                     name='Real Wage (%)',
                     marker_color=[color_positive if value >= 0 else color_negative for value in df['Real Average Wage % change']]))

# Customize hover text
hover_text = [f'{val:.2f}%' for val in df['Real Average Wage % change']]
fig.update_traces(hovertext=hover_text, hoverinfo='text')


# Update layout
fig.update_layout(
    title='Real Average Wage % (1992-2023)',
    xaxis_title='Year',
    yaxis_title='% change',
    plot_bgcolor=color_background,
    paper_bgcolor=color_background,
    xaxis=dict(
        dtick=2
    ),  # Adjust the tick frequency here (e.g., 1 for every year)
)

# Show the plot
fig.show()


   Year  Average wage (RON)  Inflation   CPI-1991  CPI-2023  \
0  1991            0.764775      170.2   1.000000  0.000461   
1  1992            2.017233      210.4   3.104000  0.001430   
2  1993            6.110217      256.1  11.053344  0.005092   
3  1994           14.011267      136.7  26.163265  0.012053   
4  1995           21.562483       32.3  34.614000  0.015947   

   Real Average Wage (RON) - 2023 prices  Real Average Wage % change  
0                                   1660                         NaN  
1                                   1410                      -15.06  
2                                   1199                      -14.96  
3                                   1162                       -3.09  
4                                   1352                       16.35  
